# Segmenting and Clustering Neighborhoods in Toronto

### Libraries

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1. Data scrapping

In [4]:

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify()) # Display the structure of the webpage
table = soup.find('table',{'class':'wikitable sortable'})
header = [th.text.rstrip() for th in table.find_all('th')]

cell1 = []; cell2 = []; cell3 = []

for row in table.findAll('tr'):
    cells = [td.text.rstrip() for td in row.find_all('td')]
    if len(cells)==3: # skip header
        cell1.append(cells[0])
        cell2.append(cells[1])
        cell3.append(cells[2])
            
df = pd.DataFrame(columns = header)
df[header[0]] = cell1
df[header[1]] = cell2
df[header[2]] = cell3
df = df[df['Borough']!='Not assigned'].reset_index(drop=True)
#df = df.groupby(['Postal Code','Borough']).agg({'Neighbourhood':','.join})  

df['Neighborhood']=df['Neighbourhood']

display(df)
print(df.shape[0])

,Postal Code,Borough,Neighbourhood,Neighborhood
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North","The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...","Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...","Old Mill South, King's Mill Park, Sunnylea, Hu..."


103


### 2. Latitude and Longitude

Let's get the geographical coordinates of Toronto.

In [5]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Unable to get the geographical coordinates of the neighborhoods using the Geocoder package. Use of the csv file instead.

In [6]:
file = 'Geospatial_Coordinates.csv'
df_coor = pd.read_csv(file)
df_merged = pd.merge(df_coor, df, on='Postal Code')

df_merged.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge","Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek","Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill","Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae,Cedarbrae


 We decide to work with only boroughs that contain the word Toronto.

In [7]:
toronto_data = df_merged[df_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood,Neighborhood
0,M4E,43.676357,-79.293031,East Toronto,The Beaches,The Beaches
1,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale","The Danforth West, Riverdale"
2,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West","India Bazaar, The Beaches West"
3,M4M,43.659526,-79.340923,East Toronto,Studio District,Studio District
4,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,Lawrence Park


In [8]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3. Clustering

Foursquare Credentials and Version

In [9]:
CLIENT_ID = '-' # your Foursquare ID
CLIENT_SECRET = '-' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: -
CLIENT_SECRET:-


In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

To do not share the Foursquare credentials, the data have been saved and then loaded again:

In [11]:
#toronto_venues.to_csv('Toronto_venues.csv')
toronto_venues = pd.read_csv('Toronto_venues.csv')
toronto_venues

,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,1,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,2,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
3,3,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
4,4,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
...,...,...,...,...,...,...,...,...
3171,3171,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Measurement Park,43.666916,-79.312631,Park
3172,3172,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Breakfast Club,43.662811,-79.310174,Breakfast Spot
3173,3173,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Carters Landing,43.662414,-79.309898,Bistro
3174,3174,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Country Style,43.667662,-79.312006,Coffee Shop


### Analyze Each Neighborhood

In [12]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [13]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,Berczy Park,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.00,0.010000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.01,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.00,0.00,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.00,0.071429,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
4,Central Bay Street,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.00,0.01,0.020000,0.000000,0.00,0.000000,0.00,0.010000,0.020000
5,Christie,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.00,0.020000,0.000000,0.00,0.010000,0.00,0.010000,0.000000
6,Church and Wellesley,0.00,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,...,0.00,0.00,0.00,0.010000,0.000000,0.00,0.010000,0.00,0.000000,0.020000
7,"Commerce Court, Victoria Hotel",0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.00,0.020000,...,0.00,0.00,0.00,0.020000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
8,Davisville,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.010000,0.010000,0.00,0.010000,0.00,0.010000,0.010000
9,Davisville North,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.010000,0.010000,0.00,0.010000,0.00,0.010000,0.020000


Run _k_-means to cluster the neighborhood into 10 clusters.

In [14]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 8, 9, 2, 1, 8, 1, 0, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Restaurant,Beer Bar,Bakery,Gym,Gastropub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Furniture / Home Store,Bakery,Restaurant,Gift Shop,Tibetan Restaurant,Arts & Crafts Store,Park
2,"Business reply mail Processing Centre, South C...",Park,Coffee Shop,Pizza Place,Brewery,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Bakery,Flea Market,Juice Bar
3,"CN Tower, King and Spadina, Railway Lands, Har...",Harbor / Marina,Coffee Shop,Café,Scenic Lookout,Sculpture Garden,Park,Track,Dog Run,Garden,Dance Studio
4,Central Bay Street,Coffee Shop,Hotel,Café,Park,Japanese Restaurant,Ramen Restaurant,Bubble Tea Shop,Breakfast Spot,Cosmetics Shop,Department Store


In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Latitude,Longitude,Borough,Neighbourhood,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,43.676357,-79.293031,East Toronto,The Beaches,The Beaches,1,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Japanese Restaurant,Beach,Grocery Store,Bakery,Sandwich Place,Caribbean Restaurant
1,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale","The Danforth West, Riverdale",1,Greek Restaurant,Coffee Shop,Café,Pub,Italian Restaurant,Bank,Discount Store,Spa,Restaurant,Ramen Restaurant
2,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West","India Bazaar, The Beaches West",9,Indian Restaurant,Coffee Shop,Grocery Store,Park,Restaurant,Beach,Brewery,Bakery,Butcher,Café
3,M4M,43.659526,-79.340923,East Toronto,Studio District,Studio District,1,Coffee Shop,Brewery,American Restaurant,Vietnamese Restaurant,Bakery,Bar,Diner,Café,French Restaurant,Sushi Restaurant
4,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,Lawrence Park,5,Park,Bookstore,Coffee Shop,College Gym,Trail,College Quad,Café,Gym / Fitness Center,Event Space,Ethiopian Restaurant


In [17]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters